### Briefly filterting the data

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [57]:
df = pd.read_excel(R"C:\Users\tripl\Downloads\Data_Entry_2017_v2020_Excel_Format.xlsx")
#All scans that contain multilple labels are removed, not all data will be used for training and it makes sense to 
#have clearly defined labels for the data used for training
df = df[~df['Finding Labels'].str.contains("\|")] 

<>:4: DeprecationWarning: invalid escape sequence \|
<>:4: DeprecationWarning: invalid escape sequence \|
C:\Users\tripl\AppData\Local\Temp\ipykernel_19284\1032160806.py:4: DeprecationWarning: invalid escape sequence \|
  df = df[~df['Finding Labels'].str.contains("\|")]


In [58]:
df

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
0,00000001_000.png,Cardiomegaly,0,1,57,M,PA,2682,2749,0.143,0.143
3,00000002_000.png,No Finding,0,2,80,M,PA,2500,2048,0.171,0.171
4,00000003_001.png,Hernia,0,3,74,F,PA,2500,2048,0.168,0.168
5,00000003_002.png,Hernia,1,3,75,F,PA,2048,2500,0.168,0.168
7,00000003_004.png,Hernia,3,3,77,F,PA,2500,2048,0.168,0.168
...,...,...,...,...,...,...,...,...,...,...,...
112114,00030801_000.png,No Finding,0,30801,38,M,PA,2500,2048,0.168,0.168
112116,00030802_000.png,No Finding,0,30802,28,M,PA,2048,2500,0.168,0.168
112117,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,0.168
112118,00030804_000.png,No Finding,0,30804,29,F,PA,2048,2500,0.168,0.168


In [59]:
#Total number of occurences for each label is calcualted 
place_holder_dataframe = {'Total Number': [], 'Label': []}
counting_labels = pd.DataFrame(place_holder_dataframe)

for i in df["Finding Labels"].unique():
    new_row = {'Total Number': df[df["Finding Labels"] == i]["Finding Labels"].count(), 'Label': i}
    new_row_df = pd.DataFrame(new_row, index=[0])
    counting_labels = pd.concat([counting_labels, new_row_df], ignore_index=True)

In [60]:
counting_labels.sort_values(by='Total Number', ascending=False)

,Total Number,Label
1,60361.0,No Finding
3,9547.0,Infiltration
7,4215.0,Atelectasis
6,3955.0,Effusion
4,2705.0,Nodule
9,2194.0,Pneumothorax
8,2139.0,Mass
12,1310.0,Consolidation
10,1126.0,Pleural_Thickening
0,1093.0,Cardiomegaly


In [61]:
#The dataset is too large to be loaded into memory at once, so when the models are trained and tested a batch of data is loaded
#into memory. This batch of data has to contain all 14 classes at least once. A batch size of 500 was chosen based on 
#prior testing, this search was however not exhaustive.  
#The above calculated total number of labels will be used to caculte a value that will represent how many times each label
#should be sampled into a batch of size 500

In [62]:
#Dictionary containing these values is stored so that it can later be used in a loop to sample the original dataset to create
#a filtered dataset

value_dictionary = {}

try:
    #The Hernia class is dropped due to the lack of labels
    counting_labels = counting_labels.drop(2)
except:
    print("Column already dropped")

for index, row in counting_labels.iterrows():
    dictionary_value = row["Total Number"]/500
    #value_dictionary[row["Label"]] = round(row["Total Number"]/500)
    value_dictionary[row["Label"]] = round(dictionary_value)
    

In [63]:
value_dictionary

{'Cardiomegaly': 2,
 'No Finding': 121,
 'Infiltration': 19,
 'Nodule': 5,
 'Emphysema': 2,
 'Effusion': 8,
 'Atelectasis': 8,
 'Mass': 4,
 'Pneumothorax': 4,
 'Pleural_Thickening': 2,
 'Fibrosis': 1,
 'Consolidation': 3,
 'Edema': 1,
 'Pneumonia': 1}

In [64]:
#Each label is now given it's own unique colum

headings = ['Cardiomegaly', 'No Finding',  'Infiltration', 'Nodule',
            'Emphysema', 'Effusion', 'Atelectasis', 'Mass', 'Pneumothorax',
            'Pleural_Thickening', 'Fibrosis', 'Consolidation', 'Edema',
            'Pneumonia']

for heading in headings:
    df[heading] = 0  # Initialize new column with zeros

In [65]:
#If a column name is present in the Finding Labels value for that row, that column's value is set to 1 instead of 0

for index, row in df.iterrows():
    labels = row['Finding Labels'].split('|')  
    for label in labels:
        if label in headings:  # checking if the label is a column name
            df.loc[index, label] = 1  # updating the corresponding cell value to 1

In [66]:
import pandas as pd

#Data is now encoded in a list with a format of [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] depending which label has the value 1

columns_list = ['Cardiomegaly', 'No Finding', 'Infiltration', 'Nodule',
            'Emphysema', 'Effusion', 'Atelectasis', 'Mass', 'Pneumothorax',
            'Pleural_Thickening', 'Fibrosis', 'Consolidation', 'Edema',
            'Pneumonia']

# Create a new column 'Labels' that will be a list of values from these columns
df['Labels'] = df[columns_list].values.tolist()

In [67]:
df

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,...,Effusion,Atelectasis,Mass,Pneumothorax,Pleural_Thickening,Fibrosis,Consolidation,Edema,Pneumonia,Labels
0,00000001_000.png,Cardiomegaly,0,1,57,M,PA,2682,2749,0.143,...,0,0,0,0,0,0,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,00000002_000.png,No Finding,0,2,80,M,PA,2500,2048,0.171,...,0,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,00000003_001.png,Hernia,0,3,74,F,PA,2500,2048,0.168,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
5,00000003_002.png,Hernia,1,3,75,F,PA,2048,2500,0.168,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
7,00000003_004.png,Hernia,3,3,77,F,PA,2500,2048,0.168,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112114,00030801_000.png,No Finding,0,30801,38,M,PA,2500,2048,0.168,...,0,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
112116,00030802_000.png,No Finding,0,30802,28,M,PA,2048,2500,0.168,...,0,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
112117,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,...,0,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
112118,00030804_000.png,No Finding,0,30804,29,F,PA,2048,2500,0.168,...,0,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


### Creating the filtered dataframe with shuffled rows

In [68]:
#.append arguemnt was causing an error message to appear, this simply stated that .append would be removed in a future pandas
#upodate, this can be replaced with .concat but would need some slight modifications to the code below
import warnings
warnings.filterwarnings("ignore")


#The dataframe_dictionary will contain a filtered version of the df dataframe for each class, this will be used to check if 
#each class still has a row to append, as after a row is append to the updated dataframe it's removed from the dataframe in the
#dataframe_dictionary

dataframe_dictionary = {}

for value in columns_list:
    dataframe_dictionary[value] = df[df["Finding Labels"] == value]
    
    
#updated_dataframe is the new fileterd dataframe where every 500 rows each class appears at least once
    
updated_dataframe = pd.DataFrame()

#The Pneumonia class is the first to run out of rows, once this runs out of rows the updated_dataframe stops appending new rows
while len(dataframe_dictionary["Pneumonia"]) != 0 :
    for value in value_dictionary:
        if len(dataframe_dictionary["Pneumonia"]) == 0:
            break
        i = 0
        while i < value_dictionary[value]:
            if len(dataframe_dictionary["Pneumonia"]) == 0:
                break
            try:
                #dataframe stored in the dataframe_dictionary is accessed for the label currently looping through the loop
                #The first occurence of that label is taken and appened to updated_dataframe and is dropped from the dataframe
                #it was taken from and the index is reset 
                index = dataframe_dictionary[value].index[0]
                row_to_append = dataframe_dictionary[value].iloc[0]
                updated_dataframe = updated_dataframe.append(row_to_append)
                dataframe_dictionary[value] = dataframe_dictionary[value].drop(index)
                dataframe_dictionary[value] = dataframe_dictionary[value].reset_index(drop=True)
                i += 1
            except IndexError:
                i += 1
                print("Index Error", value)
    print(len(updated_dataframe))    
    

warnings.filterwarnings("default")

181
362
543
724
905
1086
1267
1448
1629
1810
1991
2172
2353
2534
2715
2896
3077
3258
3439
3620
3801
3982
4163
4344
4525
4706
4887
5068
5249
5430
5611
5792
5973
6154
6335
6516
6697
6878
7059
7240
7421
7602
7783
7964
8145
8326
8507
8688
8869
9050
9231
9412
9593
9774
9955
10136
10317
10498
10679
10860
11041
11222
11403
11584
11765
11946
12127
12308
12489
12670
12851
13032
13213
13394
13575
13756
13937
14118
14299
14480
14661
14842
15023
15204
15385
15566
15747
15928
16109
16290
16471
16652
16833
17014
17195
17376
17557
17738
17919
18100
18281
18462
18643
18824
19005
19186
19367
19548
19729
19910
20091
20272
20453
20634
20815
20996
21177
21358
21539
21720
21901
22082
22263
22444
22625
22806
22987
23168
23349
23530
23711
23892
24073
24254
24435
24616
24797
24978
25159
25340
25521
25702
25883
26064
26245
26426
26607
26788
26969
27150
27331
27512
27693
27874
28055
28236
28417
28598
28779
28960
29141
29322
29503
29684
29865
30046
30227
30408
30589
30770
30951
31132
31313
31494
31675
31856
3203

In [69]:
updated_dataframe.reset_index(drop=True)

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,...,Effusion,Atelectasis,Mass,Pneumothorax,Pleural_Thickening,Fibrosis,Consolidation,Edema,Pneumonia,Labels
0,00000001_000.png,Cardiomegaly,0,1,57,M,PA,2682,2749,0.143,...,0,0,0,0,0,0,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,00000008_000.png,Cardiomegaly,0,8,68,F,PA,2048,2500,0.171,...,0,0,0,0,0,0,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,00000002_000.png,No Finding,0,2,80,M,PA,2500,2048,0.171,...,0,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,00000005_000.png,No Finding,0,5,69,F,PA,2048,2500,0.168,...,0,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,00000005_001.png,No Finding,1,5,69,F,AP,2500,2048,0.168,...,0,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58277,00019426_002.png,Consolidation,2,19426,47,F,PA,2544,3056,0.139,...,0,0,0,0,0,0,1,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
58278,00019538_003.png,Consolidation,3,19538,14,M,AP,2544,3056,0.139,...,0,0,0,0,0,0,1,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
58279,00019550_012.png,Consolidation,6,19550,9,F,AP,2048,2500,0.168,...,0,0,0,0,0,0,1,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
58280,00014904_003.png,Edema,1,14904,33,F,AP,2500,2048,0.168,...,0,0,0,0,0,0,0,1,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]"


In [70]:
updated_dataframe.to_csv("Filterd_Data_14_Class_Chest X_Ray14")

### Calculating total number of labels now

In [72]:
df = updated_dataframe.copy()

In [73]:
place_holder_dataframe = {'Total Number': [], 'Label': []}
counting_labels = pd.DataFrame(place_holder_dataframe)

for i in df["Finding Labels"].unique():
    new_row = {'Total Number': df[df["Finding Labels"] == i]["Finding Labels"].count(), 'Label': i}
    new_row_df = pd.DataFrame(new_row, index=[0])
    counting_labels = pd.concat([counting_labels, new_row_df], ignore_index=True)

In [74]:
df_sorted = counting_labels.sort_values(by='Total Number', ascending=False)

In [75]:
df_sorted

,Total Number,Label
1,38962.0,No Finding
2,6118.0,Infiltration
5,2576.0,Effusion
6,2576.0,Atelectasis
3,1610.0,Nodule
7,1288.0,Mass
8,1288.0,Pneumothorax
11,966.0,Consolidation
0,644.0,Cardiomegaly
4,644.0,Emphysema
